In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
import matplotlib
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

Categorizing a dataset and applying a function to each group, is often a critical component of a data analysis workflow.  

After loading, merging, and preparing a dataset, we will turn to compute statistics or make *pivot tables* for reporting or visulization purposes.

Contents in this chapter:  
- Split a pandas object into pieces using one or more keys.
- Calculate group summary statistics, with basic functions or **user-defined** functions.
- Apply within-group transformations.
- Compute pivot tables and cross-tabulations.
- Perform quantile analysis and other satistical group analyses.

# GroupBy Mechanics

Basic method: **Split-Apply-Combine**  
- Split data by one or more keys on particular axis.
- Apply the splitted data by groups to produce new values.
- Combine the applied data into a result object.

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


Compute the `mean` of the `data1` column by the label from `key1`.

Methods:  Access `data1` and call `groupby` with the column at `key1`.

In [3]:
grouped = df['data1'].groupby(df['key1'])  # Saved as a pandas GroupBy object.
grouped.mean()  # Call a method of the GroupBy object.

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

Frequently, the grouping information can be found in the same DataFrame as the data we want to work on. In that case we can directly pass column names as the group keys.

In [4]:
df.groupby('key1').mean()

C:\Users\ZCHDesktop\AppData\Local\Temp\ipykernel_4836\2057825055.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('key1').mean()


,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


Pass multiple arrays, and direct method.

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [6]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


All Series with matched length can be valid group keys.  
In the example below, we use two Series which are irrelevant with the original DataFrame `df` for grouping.

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

Size of a GroupBy object.  
**Any missing values in a group key will be excluded from the result.**

In [9]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterating Over Groups

Print all elements by grouping.

In [10]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023


In [11]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023


Turn a GroupBy object to a dictionary to select pieces.

In [12]:
pieces = df.groupby('key1')
pieces

In [13]:
list(pieces)

[('a',
    key1 key2     data1     data2
  0    a  one -0.204708  1.393406
  1    a  two  0.478943  0.092908
  4    a  one  1.965781  1.246435),
 ('b',
    key1 key2     data1     data2
  2    b  one -0.519439  0.281746
  3    b  two -0.555730  0.769023)]

In [14]:
dict(list(pieces))

{'a':   key1 key2     data1     data2
 0    a  one -0.204708  1.393406
 1    a  two  0.478943  0.092908
 4    a  one  1.965781  1.246435,
 'b':   key1 key2     data1     data2
 2    b  one -0.519439  0.281746
 3    b  two -0.555730  0.769023}

In [15]:
dict(list(pieces))['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


By default `groupby` groups on `axis=0`, but we can group by other axes, such as group the columns.  
We can also iterate the group as we did before.

In [16]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## Selecting a Column or Subset of Columns

Indexing a `GroupBy` object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:  
- `df.groupby('key1')['data1']` == `df['data1'].groupby(df['key1'])`
- `df.groupby('key1')[['data2']]` == `df[['data2']].groupby(df['key1'])`

**Pay attension:**  
The object returned by this index operation is a grouped DataFrame if a list or array is passed or a grouped Series if only a single column name is passed as a scalar, pay attension to the differences between 2 examples below.

In [18]:
# Return a DataFrame with column name 'data2'.

df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [19]:
# Return a multi-indexed Series with no column name.

df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

## Grouping with Dicts and Series

Grouping information may exist in a object other than an array.

In [20]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan  # Add a few NA values.

people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


Suppose we have a group correspondence for the columns and want to sum together the columns by group.

This kind of group correspondence can be a `dict`, a `Series` or others.  
For example, in the example below, we use a `dict` as a mapping. Equivalently, we can turn the `dict` into a `Series` and pass the group key as the new `Series`, we can see that it reachs the same effect.

In [21]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)

by_column.sum()

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


In [22]:
map_series = pd.Series(mapping)

by_column = people.groupby(map_series, axis=1)
by_column.count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Grouping with Functions

Using Python functions is a more generic way of defining a group mapping compared with a dict or Series. **Any function passed as a group key will be called once per index value, with the return values being used as the group names.**

We use the example DataFrame from the previous section to explain that.

In [23]:
# Group along axis=0, if we pass axis=1, we can expect to get a DataFrame with only 1 column,
# and the col_name is 1, because the col_name length of original DataFrame is 1 (a, b, c, d, e).

people.groupby(len).sum()

,a,b,c,d,e
3,0.591569,-0.993608,0.798764,-0.791374,2.119639
5,0.886429,-2.001637,-0.371843,1.669025,-0.438570
6,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


We can also mix different kinds of group keys.

In [24]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.539741 -1.296221  0.274992 -1.021228 -0.577087
  two  0.124121  0.302614  0.523772  0.000940  1.343810
5 one  0.886429 -2.001637 -0.371843  1.669025 -0.438570
6 two -0.713544 -0.831154 -2.370232 -1.860761 -0.860757

## Grouping by Index Levels

For a `MultiIndex` object, we can group using one of the levels of an axis index.

In [25]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.560145 -1.265934  0.119827 -1.063512  0.332883
1     -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
2      0.286350  0.377984 -0.753887  0.331286  1.349742
3      0.069877  0.246674 -0.011862  1.004812  1.327195

Pass the level number of name using the `level` keyword to group with a hierarchy index.

In [26]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [27]:
hier_df.groupby(level=1, axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


# Data Aggregation

Aggeregations refer to any data transformation that produces scalar values from arrays.  
Some common aggregations including `count`, `sum`, `mean`, `median`, `std`, `var`, `min`, `max`, `prod`, `first`, `last` and so on. We can also use our own devising and additionally call any method that is also defined on the grouped object.

In [28]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [29]:
grouped = df.groupby('key1')

grouped['data1'].quantile(0.9)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [30]:
# Use our own aggregation functions.

def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

C:\Users\ZCHDesktop\AppData\Local\Temp\ipykernel_4836\3470737030.py:6: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [31]:
grouped.describe().stack()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean   0.746672  0.910916
     std    1.109736  0.712217
     min   -0.204708  0.092908
     25%    0.137118  0.669671
     50%    0.478943  1.246435
     75%    1.222362  1.319920
     max    1.965781  1.393406
b    count  2.000000  2.000000
     mean  -0.537585  0.525384
     std    0.025662  0.344556
     min   -0.555730  0.281746
     25%   -0.546657  0.403565
     50%   -0.537585  0.525384
     75%   -0.528512  0.647203
     max   -0.519439  0.769023

## Column-Wise and Multiple Function Application

In [32]:
path = r"D:\Projects\PyLearning\Exercise\py_for_data_analysis\examples\tips.csv"
tips = pd.read_csv(path)

tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
